In [1]:
from utils_kdd99 import *
data_x, data_y = load_data(use_full_dataset=True)

# 4つのクラスラベルに変換する
data_y = data_y.map(lambda x: attack_label_class[x])
data_y.value_counts()


特徴量の数：41
各特徴量の名前：duration, protocol_type, service, flag, src_bytes, dst_bytes, land, wrong_fragment, urgent, hot, num_failed_logins, logged_in, num_compromised, root_shell, su_attempted, num_root, num_file_creations, num_shells, num_access_files, num_outbound_cmds, is_host_login, is_guest_login, count, srv_count, serror_rate, srv_serror_rate, rerror_rate, srv_rerror_rate, same_srv_rate, diff_srv_rate, srv_diff_host_rate, dst_host_count, dst_host_srv_count, dst_host_same_srv_rate, dst_host_diff_srv_rate, dst_host_same_src_port_rate, dst_host_srv_diff_host_rate, dst_host_serror_rate, dst_host_srv_serror_rate, dst_host_rerror_rate, dst_host_srv_rerror_rate
smurf              2807886
neptune            1072017
normal              972781
satan                15892
ipsweep              12481
portsweep            10413
nmap                  2316
back                  2203
warezclient           1020
teardrop               979
pod                    264
guess_passwd            53
buffer_overflo

dos       3883370
normal     972781
probe       41102
r2l          1126
u2r            52
Name: true_label, dtype: int64

In [2]:
from sklearn.model_selection import train_test_split
# data_y = pd.get_dummies(data_y)
# データを分割する。テストと，学習の比は1：2なるようにします。
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.33, random_state=RANDOM_SEED, stratify=data_y)
print(f"x_train: {len(x_train)}, x_test: {len(x_test)}")
print(f'y_train')
y_train.value_counts()


x_train: 3281948, x_test: 1616483
y_train


dos       2601857
normal     651763
probe       27538
r2l           755
u2r            35
Name: true_label, dtype: int64

In [3]:
print(f'y_test')
y_test.value_counts()


y_test


dos       1281513
normal     321018
probe       13564
r2l           371
u2r            17
Name: true_label, dtype: int64

In [4]:
# 目的変数を数値に変換
correspondences = {
    'dos': 0,
    'normal': 1,
    'probe': 2,
    'r2l': 3,
    'u2r': 4
}
y_train = y_train.map(lambda x: correspondences[x])
y_test = y_test.map(lambda x: correspondences[x])
y_train.head()

1537115    0
4657510    0
1894345    0
1649777    0
1634756    0
Name: true_label, dtype: int64

In [5]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

# LightGBM parameters
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class': 5,
        'metric': {'multi_error'}, # 評価指標 : 誤り率(= 1-正答率)  another multi_logloss
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 1000, #1000回学習
        'verbose': 0
}

# モデルの学習
model = lgb.train(params, # パラメータ
            train_set=lgb_train, # トレーニングデータの指定
            valid_sets=lgb_eval, # 検証データの指定
            callbacks=[lgb.early_stopping(100)]
               )

# テストデータの予測 (クラス1の予測確率(クラス1である確率)を返す)
y_pred_prob = model.predict(x_test)
# テストデータの予測 (予測クラスを返す)
y_pred = np.argmax(y_pred_prob, axis=1) # 一番大きい予測確率のクラスを予測クラスに
y_pred = pd.Series(y_pred)
y_pred.value_counts()

/Users/satun/anaconda3/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.129848 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

0    1281353
1     320899
2      13510
3        482
4        239
dtype: int64

In [6]:
print(classification_report(y_test, y_pred, target_names=correspondences.keys()))

              precision    recall  f1-score   support

         dos       1.00      1.00      1.00   1281513
      normal       1.00      1.00      1.00    321018
       probe       1.00      0.99      0.99     13564
         r2l       0.73      0.95      0.83       371
         u2r       0.00      0.00      0.00        17

    accuracy                           1.00   1616483
   macro avg       0.75      0.79      0.76   1616483
weighted avg       1.00      1.00      1.00   1616483



In [7]:
for key, confusion_matrix in zip(correspondences.keys(), multilabel_confusion_matrix(y_test, y_pred)):
    print(f"{key}    TP: {confusion_matrix[1][1]}, TN: {confusion_matrix[0][0]}, FP: {confusion_matrix[1][0]}, FN: {confusion_matrix[0][1]}")


dos    TP: 1281328, TN: 334945, FP: 185, FN: 25
normal    TP: 320603, TN: 1295169, FP: 415, FN: 296
probe    TP: 13454, TN: 1602863, FP: 110, FN: 56
r2l    TP: 354, TN: 1615984, FP: 17, FN: 128
u2r    TP: 0, TN: 1616227, FP: 17, FN: 239


In [8]:
model = keras.Sequential([
    Dense(units=19, activation='relu', input_dim=38, name='encoder1'),
    Dense(units=10, activation='relu', name='encoder2'),
    Dense(units=19, activation='relu'),
    Dense(units=38, activation='relu'),
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder1 (Dense)            (None, 19)                741       
                                                                 
 encoder2 (Dense)            (None, 10)                200       
                                                                 
 dense (Dense)               (None, 19)                209       
                                                                 
 dense_1 (Dense)             (None, 38)                760       
                                                                 
Total params: 1,910
Trainable params: 1,910
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(x_train, x_train,
          epochs=1, # データセットを使って学習する回数
        batch_size=32,
        shuffle=True, # データをシャッフルすることで，．
        validation_data=(x_train, x_train), # 評価用データ（検証データ）の指定
        verbose=1,
        use_multiprocessing=True
          )


2023-07-18 09:54:23.203821: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


102561/102561 [==============================] - 89s 860us/step - loss: 0.6288 - accuracy: 0.1961 - val_loss: 0.6018 - val_accuracy: 0.2068


In [10]:
x_pred = model.predict(x_test)


50516/50516 [==============================] - 13s 255us/step


In [11]:
encoder = keras.Sequential([model.get_layer('encoder1'),
                            model.get_layer('encoder2')])


In [12]:
columns = list(map(lambda x: 'feature' + str(x), range(10)))
x_train_encoded = pd.DataFrame(data=encoder.predict(x_train), index=x_train.index, columns=columns)
x_test_encoded = pd.DataFrame(data=encoder.predict(x_test), index=x_test.index, columns=columns)
x_train_new_feature = x_train.merge(x_train_encoded, right_index=True, left_index=True)
x_test_new_feature = x_test.merge(x_test_encoded, right_index=True, left_index=True)
x_train_new_feature


50516/50516 [==============================] - 12s 232us/step


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9
1537115,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,1.485163,0.573950,0.0,0.000000,0.336887,0.000000,0.292402,0.501981,1.166986,0.499266
4657510,-0.066833,-0.001949,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,0.000000,5.674854,0.0,5.825389,4.213655,14.140607,7.169546,5.212486,0.163113,1.542490
1894345,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,1.486886,0.573789,0.0,0.000000,0.336487,0.000000,0.293716,0.501628,1.167947,0.499632
1649777,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,1.485163,0.573950,0.0,0.000000,0.336887,0.000000,0.292402,0.501981,1.166986,0.499266
1634756,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,1.485163,0.573950,0.0,0.000000,0.336887,0.000000,0.292402,0.501981,1.166986,0.499266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391255,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,1.485163,0.573950,0.0,0.000000,0.336887,0.000000,0.292402,0.501981,1.166986,0.499266
2471385,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,1.486886,0.573789,0.0,0.000000,0.336487,0.000000,0.293716,0.501628,1.167947,0.499632
2676542,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,1.485163,0.573950,0.0,0.000000,0.336887,0.000000,0.292402,0.501981,1.166986,0.499266
1475808,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,1.485163,0.573950,0.0,0.000000,0.336887,0.000000,0.292402,0.501981,1.166986,0.499266


In [13]:
lgb_train = lgb.Dataset(x_train_new_feature, y_train)
lgb_eval = lgb.Dataset(x_test_new_feature, y_test, reference=lgb_train)

# LightGBM parameters
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class': 5,
        'metric': {'multi_error'}, # 評価指標 : 誤り率(= 1-正答率)  another multi_logloss
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 1000, #1000回学習
        'verbose': 0
}

# モデルの学習
model = lgb.train(params, # パラメータ
            train_set=lgb_train, # トレーニングデータの指定
            valid_sets=lgb_eval, # 検証データの指定
            callbacks=[lgb.early_stopping(100)]
               )

# テストデータの予測 (クラス1の予測確率(クラス1である確率)を返す)
y_pred_prob = model.predict(x_test_new_feature)
# テストデータの予測 (予測クラス(0 or 1 or 2)を返す)
y_pred = np.argmax(y_pred_prob, axis=1) # 一番大きい予測確率のクラスを予測クラスに
y_pred = pd.Series(y_pred)
y_pred.value_counts()


/Users/satun/anaconda3/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.256004 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

0    1281322
1     321171
2      13465
3        345
4        180
dtype: int64

In [14]:
print(classification_report(y_test, y_pred, target_names=correspondences.keys()))


              precision    recall  f1-score   support

         dos       1.00      1.00      1.00   1281513
      normal       1.00      1.00      1.00    321018
       probe       1.00      0.99      0.99     13564
         r2l       0.95      0.89      0.92       371
         u2r       0.00      0.00      0.00        17

    accuracy                           1.00   1616483
   macro avg       0.79      0.78      0.78   1616483
weighted avg       1.00      1.00      1.00   1616483



In [15]:
for key, confusion_matrix in zip(correspondences.keys(), multilabel_confusion_matrix(y_test, y_pred)):
    print(f"{key}    TP: {confusion_matrix[1][1]}, TN: {confusion_matrix[0][0]}, FP: {confusion_matrix[1][0]}, FN: {confusion_matrix[0][1]}")


dos    TP: 1281287, TN: 334935, FP: 226, FN: 35
normal    TP: 320878, TN: 1295172, FP: 140, FN: 293
probe    TP: 13418, TN: 1602872, FP: 146, FN: 47
r2l    TP: 329, TN: 1616096, FP: 42, FN: 16
u2r    TP: 0, TN: 1616286, FP: 17, FN: 180


In [16]:
model = keras.Sequential([
    Dense(units=10, activation='relu', input_dim=38, name='encoder1'),
    Dense(units=5, activation='relu', name='encoder2'),
    Dense(units=10, activation='relu'),
    Dense(units=38, activation='relu'),
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder1 (Dense)            (None, 10)                390       
                                                                 
 encoder2 (Dense)            (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 10)                60        
                                                                 
 dense_3 (Dense)             (None, 38)                418       
                                                                 
Total params: 923
Trainable params: 923
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(x_train, x_train,
          epochs=1, # データセットを使って学習する回数
        batch_size=32,
        shuffle=True, # データをシャッフルすることで，．
        validation_data=(x_train, x_train), # 評価用データ（検証データ）の指定
        verbose=1,
        use_multiprocessing=True
          )


102561/102561 [==============================] - 118s 1ms/step - loss: 0.6666 - accuracy: 0.7483 - val_loss: 0.6441 - val_accuracy: 0.7767


In [18]:
encoder = keras.Sequential([model.get_layer('encoder1'),
                            model.get_layer('encoder2')])


In [19]:
columns = list(map(lambda x: 'feature' + str(x), range(5)))
x_train_encoded = pd.DataFrame(data=encoder.predict(x_train), index=x_train.index, columns=columns)
x_test_encoded = pd.DataFrame(data=encoder.predict(x_test), index=x_test.index, columns=columns)
x_train_new_feature = x_train.merge(x_train_encoded, right_index=True, left_index=True)
x_test_new_feature = x_test.merge(x_test_encoded, right_index=True, left_index=True)
x_train_new_feature


50516/50516 [==============================] - 12s 228us/step


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,feature0,feature1,feature2,feature3,feature4
1537115,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,0.000000,0.564129,3.360172,0.161625,2.288970
4657510,-0.066833,-0.001949,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,4.079245,4.079791,12.364801,6.891846,8.588043,0.000000,10.471345
1894345,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,0.000000,0.564913,3.360382,0.160949,2.289763
1649777,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,0.000000,0.564129,3.360172,0.161625,2.288970
1634756,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,0.000000,0.564129,3.360172,0.161625,2.288970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391255,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,0.000000,0.564129,3.360172,0.161625,2.288970
2471385,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,0.000000,0.564913,3.360383,0.160949,2.289763
2676542,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,0.000000,0.564129,3.360172,0.161625,2.288971
1475808,-0.066833,-0.000853,-0.001696,-0.002391,-0.015139,-0.001103,-0.026521,-0.004391,-0.409368,-0.002097,...,-0.156668,-0.466405,-0.465454,-0.250832,-0.249632,0.000000,0.564129,3.360172,0.161625,2.288971


In [20]:
lgb_train = lgb.Dataset(x_train_new_feature, y_train)
lgb_eval = lgb.Dataset(x_test_new_feature, y_test, reference=lgb_train)
# モデルの学習
model = lgb.train(params, # パラメータ
            train_set=lgb_train, # トレーニングデータの指定
            valid_sets=lgb_eval, # 検証データの指定
            callbacks=[lgb.early_stopping(100)]
               )

# テストデータの予測 (クラス1の予測確率(クラス1である確率)を返す)
y_pred_prob = model.predict(x_test_new_feature)
# テストデータの予測 (予測クラス(0 or 1 or 2)を返す)
y_pred = np.argmax(y_pred_prob, axis=1) # 一番大きい予測確率のクラスを予測クラスに
y_pred = pd.Series(y_pred)
y_pred.value_counts()

/Users/satun/anaconda3/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.153446 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

0    1281300
1     321360
2      13454
3        367
4          2
dtype: int64

In [21]:
print(classification_report(y_test, y_pred, target_names=correspondences.keys()))


              precision    recall  f1-score   support

         dos       1.00      1.00      1.00   1281513
      normal       1.00      1.00      1.00    321018
       probe       1.00      0.99      0.99     13564
         r2l       0.92      0.91      0.92       371
         u2r       0.00      0.00      0.00        17

    accuracy                           1.00   1616483
   macro avg       0.78      0.78      0.78   1616483
weighted avg       1.00      1.00      1.00   1616483



In [23]:
for key, confusion_matrix in zip(correspondences.keys(), multilabel_confusion_matrix(y_test, y_pred)):
    print(f"{key}    TP: {confusion_matrix[1][1]}, TN: {confusion_matrix[0][0]}, FP: {confusion_matrix[1][0]}, FN: {confusion_matrix[0][1]}")


dos    TP: 1281282, TN: 334952, FP: 231, FN: 18
normal    TP: 320938, TN: 1295043, FP: 80, FN: 422
probe    TP: 13402, TN: 1602867, FP: 162, FN: 52
r2l    TP: 339, TN: 1616084, FP: 32, FN: 28
u2r    TP: 0, TN: 1616464, FP: 17, FN: 2


# 10％データを使用した場合
## 数値入力データのみ（38次元）の場合
dos    TP: 129147, TN: 33829, FP: 34, FN: 17
normal TP: 32045,  TN: 130870, FP: 57, FN: 55
probe  TP: 1338,   TN: 161666, FP: 17, FN: 6
r2l    TP: 353,    TN: 162637, FP: 19, FN: 18
u2r    TP: 5,      TN: 162967, FP: 12, FN: 43


## 数値入力データ+AEで学習した特徴量(10次元)（48次元）の場合
dos    TP: 129168, TN: 33813,  FP: 13, FN: 33
normal TP: 32001,  TN: 130864, FP: 101, FN: 61
probe  TP: 1334,   TN: 161658, FP: 21, FN: 14
r2l    TP: 342,    TN: 162630, FP: 30, FN: 25
u2r    TP: 4,      TN: 162965, FP: 13, FN: 45

## 数値入力データ+AEで学習した特徴量(5次元)（43次元）の場合
dos    TP: 129173, TN: 33781, FP: 8, FN: 65
normal    TP: 32018, TN: 130880, FP: 84, FN: 45
probe    TP: 1324, TN: 161659, FP: 31, FN: 13
r2l    TP: 349, TN: 162622, FP: 23, FN: 33
u2r    TP: 1, TN: 163004, FP: 16, FN: 6
